# SQL en Python
<img src=".\img\image-4.png" alt="EDA Path"
    title="A typical EDA path" width="600" height="300" />
## Conectamos con la base de datos

### Documentación de la librería SQLite3 que vamos a utilizar:
https://docs.python.org/3/library/sqlite3.html


In [1]:
# Importamos librerias
import pandas as pd
import sqlite3

/home/jose/.local/lib/python3.7/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
# Conectamos con la base de datos chinook.db
path = 'chinook.db'
connection = sqlite3.connect(path)
# Obtenemos un cursor que utilizaremos para hacer las queries
curs = connection.cursor()

In [ ]:
# Creamos una Query sencilla
query = """
SELECT *
FROM genres
"""

In [ ]:
my_query = curs.execute(query)
my_query

In [ ]:
my_query.fetchall()

In [ ]:
# Con esta función leemos los datos y lo pasamos a un DataFrame de Pandas
def sql_query(query):
    curs.execute(query)
    datos_query = curs.fetchall()
    col_names = [description[0] for description in curs.description]
    return pd.DataFrame(datos_query, columns=col_names)

In [ ]:
sql_query(query)

In [ ]:
# Tambien podemos obtener el mismo resultado directamente con pandas
pd.read_sql_query(query, connection)

## Ya podemos comenzar con la práctica de chinook:
Antes de empezar a atacar una base de datos, tendremos que saber qué hay dentro, y para ello lo mejor es ver cómo es su **modelo de datos**

![imagen](./img/chinook_data_model.png)

### 1.	Facturas de Clientes de Brasil, Nombre del cliente, Id de factura, fecha de la factura y el país de la factura

In [ ]:
query = '''
SELECT *
FROM customers as c, invoices as i
WHERE c.CustomerId = i.CustomerId
'''
pd.read_sql_query(query, connection)

In [ ]:
query = '''
SELECT c.FirstName||' '||c.LastName as 'Full Name', i.invoiceId, i.invoiceDate, i.BillingCountry
FROM customers c
JOIN invoices i ON c.CustomerId = i.CustomerId
WHERE c.Country = 'Brazil'
'''
pd.read_sql_query(query, connection)

### 2.	Facturas de Clientes de Brasil

In [ ]:
query = '''
SELECT i.*
FROM customers c
JOIN invoices i ON c.CustomerId = i.CustomerId
WHERE c.Country = 'Brazil'
'''
pd.read_sql_query(query, connection)

### 3.	Muestra cada factura asociada a cada agente de ventas con su nombre completo.

In [ ]:
query = '''
SELECT e.FirstName||' '||e.LastName as 'Full name', i.*
FROM employees e
JOIN customers c ON e.EmployeeId = c.SupportRepId
JOIN invoices i ON c.CustomerId = i.CustomerId
'''
pd.read_sql_query(query, connection)

### 4.	Para cada factura muestra el nombre del cliente, el país, el nombre del agente y el total

In [42]:
query = '''
SELECT c.FirstName||' '||c.LastName as 'Customer name', i.BillingAddress, e.FirstName||' '||e.LastName as 'Salesman name', i.total
FROM invoices i
JOIN customers c ON i.CustomerId = c.CustomerId
JOIN employees e ON c.SupportRepId = e.EmployeeId
'''
pd.read_sql_query(query, connection)

,Customer name,BillingAddress,Salesman name,Total
0,Luís Gonçalves,"Av. Brigadeiro Faria Lima, 2170",Jane Peacock,3.98
1,Luís Gonçalves,"Av. Brigadeiro Faria Lima, 2170",Jane Peacock,3.96
2,Luís Gonçalves,"Av. Brigadeiro Faria Lima, 2170",Jane Peacock,5.94
3,Luís Gonçalves,"Av. Brigadeiro Faria Lima, 2170",Jane Peacock,0.99
4,Luís Gonçalves,"Av. Brigadeiro Faria Lima, 2170",Jane Peacock,1.98
...,...,...,...,...
407,Puja Srivastava,"3,Raj Bhavan Road",Jane Peacock,5.94
408,Puja Srivastava,"3,Raj Bhavan Road",Jane Peacock,1.99
409,Puja Srivastava,"3,Raj Bhavan Road",Jane Peacock,1.98
410,Puja Srivastava,"3,Raj Bhavan Road",Jane Peacock,13.86


### 5.	Muestra cada artículo de la factura con el nombre de la canción.

In [41]:
query = '''
SELECT i.*, t.Name
FROM invoice_items i
JOIN tracks t ON i.trackid = t.trackid
'''
pd.read_sql_query(query, connection)

,InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity,Name
0,1,1,2,0.99,1,Balls to the Wall
1,2,1,4,0.99,1,Restless and Wild
2,3,2,6,0.99,1,Put The Finger On You
3,4,2,8,0.99,1,Inject The Venom
4,5,2,10,0.99,1,Evil Walks
...,...,...,...,...,...,...
2235,2236,411,3136,0.99,1,Looking For Love
2236,2237,411,3145,0.99,1,Sweet Lady Luck
2237,2238,411,3154,0.99,1,Feirinha da Pavuna/Luz do Repente/Bagaço da La...
2238,2239,411,3163,0.99,1,Samba pras moças


### 6.	Muestra todas las canciones con su nombre, formato, álbum y género.

In [49]:
query = '''
SELECT t.name as 'song name', a.title as 'album', m.name as 'formato', g.name as 'genero'
FROM tracks t
JOIN genres g ON t.genreid = g.genreid
JOIN albums a ON t.albumid = a.albumid
JOIN media_types m ON m.mediatypeid = t.mediatypeid
'''
pd.read_sql_query(query, connection)

,song name,album,formato,genero
0,For Those About To Rock (We Salute You),For Those About To Rock We Salute You,MPEG audio file,Rock
1,Balls to the Wall,Balls to the Wall,Protected AAC audio file,Rock
2,Fast As a Shark,Restless and Wild,Protected AAC audio file,Rock
3,Restless and Wild,Restless and Wild,Protected AAC audio file,Rock
4,Princess of the Dawn,Restless and Wild,Protected AAC audio file,Rock
...,...,...,...,...
3498,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,Respighi:Pines of Rome,Protected AAC audio file,Classical
3499,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",Schubert: The Late String Quartets & String Qu...,Protected AAC audio file,Classical
3500,"L'orfeo, Act 3, Sinfonia (Orchestra)",Monteverdi: L'Orfeo,Protected AAC audio file,Classical
3501,"Quintet for Horn, Violin, 2 Violas, and Cello ...",Mozart: Chamber Music,Protected AAC audio file,Classical


### 7.	Muestra cuántas canciones hay en cada playlist y el nombre de cada playlist.

In [28]:
query = '''
SELECT p.name as Nombre, COUNT(*)
FROM playlist_track pt
JOIN playlists p ON p.playlistid = pt.playlistid
GROUP BY p.playlistid
'''
pd.read_sql_query(query, connection)

,Nombre,COUNT(*)
0,Music,3290
1,TV Shows,213
2,90’s Music,1477
3,Music,3290
4,Music Videos,1
5,TV Shows,213
6,Brazilian Music,39
7,Classical,75
8,Classical 101 - Deep Cuts,25
9,Classical 101 - Next Steps,25


### 8.	Muestra cuánto ha vendido cada empleado.

In [40]:
query = '''
SELECT e.FirstName||' '||e.LastName as 'Nombre', SUM(it.quantity) as 'unidades vendidas'
FROM employees e 
JOIN customers c ON c.supportrepid = e.employeeid
JOIN invoices i ON i.customerid = c.customerid
JOIN invoice_items it ON it.invoiceid = i.invoiceid
GROUP BY e.employeeid
'''
pd.read_sql_query(query, connection)

,Nombre,unidades vendidas
0,Jane Peacock,796
1,Margaret Park,760
2,Steve Johnson,684


### 9.	¿Quién ha sido el agente de ventas que más ha vendido en 2009?

In [49]:
query = '''
SELECT e.FirstName||' '||e.LastName as 'Nombre',  SUM(it.quantity) as 'unidades_vendidas'
FROM employees e 
JOIN customers c ON c.supportrepid = e.employeeid
JOIN invoices i ON i.customerid = c.customerid
JOIN invoice_items it ON it.invoiceid = i.invoiceid
WHERE i.invoicedate LIKE '2009%'
GROUP BY e.employeeid
--Aqui va el Select en order de ejecucion
ORDER BY unidades_vendidas DESC
LIMIT 1
'''
pd.read_sql_query(query, connection)

,Nombre,unidades_vendidas
0,Steve Johnson,166


### 10.	¿Cuáles son los 3 grupos que más han vendido?

In [52]:
query = '''
SELECT t.composer as 'Artista', SUM(i.quantity) as 'Ventas'
FROM tracks t
JOIN invoice_items i ON t.trackid = i.trackid
GROUP BY t.composer
ORDER BY Ventas DESC
LIMIT 4
'''
pd.read_sql_query(query, connection)
# AQUI HAY ERROR POR EL NONE

,Artista,Ventas
0,None,596
1,Steve Harris,58
2,U2,33
3,Billy Corgan,23


### 11. Muestra cuántas canciones de Rock hay en cada playlist

In [55]:
query = '''
SELECT p.name as 'Nombre_playlist', COUNT(p.playlistid)
FROM genres g
JOIN tracks t ON g.genreid = t.genreid
JOIN playlist_track pt ON t.trackid = pt.trackid
JOIN playlists p ON pt.playlistid = p.playlistid
WHERE g.name = 'Rock'
GROUP BY p.playlistid
'''
pd.read_sql_query(query, connection)
# Esta mal por como esta hecha la tabla

,Nombre_playlist,COUNT(p.playlistid)
0,Music,1297
1,90’s Music,621
2,Music,1297
3,Grunge,14
4,Heavy Metal Classic,9


### 12. Muestra una tabla con todas canciones y su(s) Id de factura, hayan sido vendidas alguna vez o no.

In [98]:
query = '''
SELECT t.name as 'Nombre cancion', i.invoiceid as 'Id factura'
FROM tracks t
LEFT JOIN invoice_items i ON t.trackid = i.trackid
'''
pd.read_sql_query(query, connection)

,Nombre cancion,Id factura
0,For Those About To Rock (We Salute You),108.0
1,Balls to the Wall,1.0
2,Balls to the Wall,214.0
3,Fast As a Shark,319.0
4,Restless and Wild,1.0
...,...,...
3754,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",108.0
3755,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",319.0
3756,"L'orfeo, Act 3, Sinfonia (Orchestra)",NaN
3757,"Quintet for Horn, Violin, 2 Violas, and Cello ...",NaN


### 13. ¿Cuántos artistas no tienen ningún album?

In [97]:
# artists, albums
query = '''
SELECT COUNT(ar.name) as 'Artistas sin titulos'
FROM artists ar
LEFT JOIN albums al ON ar.artistid = al.artistid
WHERE al.title IS NULL 
'''
pd.read_sql_query(query, connection) 

,Artistas sin titulos
0,71
